In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import numpy as np
train_list_videos = "/data/gyuseok/labels/train_val.csv"


train_val_df = pd.read_csv(train_list_videos)
train_df, val_df = train_test_split(train_val_df, test_size = 0.1, random_state = 256)



In [2]:
feat_list, label_list = [],[]
for i in range(len(train_df)):
    video_id, label = train_df.iloc[i]
    feat_filepath = os.path.join("/data/gyuseok/pf2/", video_id + ".csv")
    if os.path.exists(feat_filepath):
        feat_list.append(np.genfromtxt(feat_filepath, delimiter=";", dtype="float"))
    else:
        feat_list.append(np.zeros(1024))
    label_list.append(int(label))


In [3]:
np.array(feat_list), np.array(label_list)

(array([[-1.10227442,  0.26739958,  0.59675986, ..., -0.36024308,
          0.20321321,  0.80227512],
        [-1.12424457,  0.72674233,  0.59265435, ...,  0.49610862,
         -0.49715522,  0.29680824],
        [-0.29612532, -0.1755002 ,  0.16692391, ...,  0.52447236,
         -0.68567502,  0.18235721],
        ...,
        [-0.29409313, -0.5019142 ,  0.74075598, ..., -0.26623023,
         -1.23138821, -0.11152034],
        [-0.05379267,  0.84707159, -0.20266861, ...,  0.62080932,
         -0.38553092, -0.20394766],
        [-0.3102265 ,  1.09195471,  0.93621373, ...,  0.78387487,
         -0.11307195, -0.57148486]]),
 array([ 8, 11, 12, ...,  0,  5, 14]))

In [4]:
np.array(feat_list).shape

(6750, 1024)

In [5]:
np.array(label_list).shape

(6750,)

In [6]:
def load_train_data(args):
    # 1. read all features in one array.
    fread = open(args.train_list_videos, "r")
    feat_list = []
    # labels are [0-9]
    label_list = []
    # load video names and events in dict
    df_videos_label = {}
    for line in open(args.train_list_videos).readlines()[1:]:
        video_id, category = line.strip().split(",")
        df_videos_label[video_id] = category

    for line in fread.readlines()[1:]:
        video_id = line.strip().split(",")[0]
        feat_filepath = os.path.join(args.feat_dir, video_id + args.feat_appendix)
        # for videos with no audio, ignored in training
        if os.path.exists(feat_filepath):
            feat_list.append(np.genfromtxt(feat_filepath, delimiter=";", dtype="float"))
            label_list.append(int(df_videos_label[video_id]))
        
    
    return np.array(feat_list), np.array(label_list)

In [7]:
def load_test_data(args):
    feat_list = []
    video_ids = []
    not_found_count = 0
    for line in open(args.test_list_videos).readlines()[1:]:
        video_id= line.strip().split(",")[0]
        video_ids.append(video_id)
        feat_filepath = os.path.join(args.feat_dir, video_id + args.file_ext)
        if not os.path.exists(feat_filepath):
            feat_list.append(np.zeros(args.feat_dim))
            not_found_count += 1
        else:
            feat_list.append(np.genfromtxt(feat_filepath, delimiter=";", dtype="float"))

    if not_found_count > 0:
        print(f'Could not find the features for {not_found_count} samples.')

    return np.array(feat_list), video_ids

In [8]:
# test
test_list_videos = "/data/gyuseok/labels/test_for_students.csv"
df = pd.read_csv(test_list_videos)
df

,Id,Category
0,LTExODM2Mzc0ODQyOTc1ODE4NDM=,NaN
1,LTUwNDU3NzgyNjE2Mzk0OTU1NjQ=,NaN
2,ODU3OTE0MDU5NzM5NDI2MDQ2,NaN
3,LTQ4OTE2NDk5ODk4MTA3NDY5Mzg=,NaN
4,MjkzNzMwNzkxNTg5NDA1Mjk3NQ==,NaN
...,...,...
744,Nzk0NzA0MjYxMTg3NDkxNjQ4OQ==,NaN
745,LTMzMjA0NjkyNzYwMDQ0NDUyNzY=,NaN
746,LTQxNDgxNjU0MzY3OTQzNzIzMTc=,NaN
747,NjQyMjk5MDM5OTgwNzY2NDU1MQ==,NaN


In [9]:
feat_list = []
video_ids = []
not_found_count = 0
for i in range(len(df)):
    video_id = df.iloc[i,0]
    video_ids.append(video_id)
    
    feat_filepath = os.path.join("/data/gyuseok/pf2/", video_id + ".csv")
    if not os.path.exists(feat_filepath):
        feat_list.append(np.zeros(1024))
        not_found_count += 1
    else:
        feat_list.append(np.genfromtxt(feat_filepath, delimiter=";", dtype="float"))

if not_found_count > 0:
    print(f'Could not find the features for {not_found_count} samples.')

In [10]:
len(feat_list)

749

In [11]:
len(video_ids)

749

In [12]:
from CustomSet import *
D_feat_dir = "/home/gyuseok/CMU/11775-hw2-handout/data/cnn3d"
dtrain_set = FeatureDataset(df, D_feat_dir)

In [13]:
dtrain_set.__getitem__(0)

(tensor([0.1166, 1.9648, 0.3180, 0.8271, 0.9208, 0.2779, 2.0496, 0.3539, 0.7603,
         0.6000, 1.9596, 0.5232, 0.6102, 0.6790, 0.9761, 1.4999, 0.0892, 1.1459,
         2.3625, 0.6248, 0.4561, 0.2311, 0.4683, 0.3226, 0.5538, 1.3529, 1.3492,
         1.3437, 0.9081, 0.5215, 0.9014, 0.7001, 1.6128, 0.6208, 1.2169, 0.6405,
         0.9080, 1.7304, 0.5166, 0.4404, 0.7629, 1.0758, 0.8774, 0.6333, 0.5127,
         0.5417, 0.9477, 0.9063, 0.1528, 0.6194, 0.9955, 0.4942, 2.7070, 0.8988,
         0.9194, 0.6058, 0.5725, 0.6293, 1.0067, 0.8443, 0.3828, 0.7803, 0.8863,
         0.6199, 1.0789, 0.3969, 0.1793, 2.1484, 0.1653, 0.5920, 0.4712, 1.3614,
         1.6496, 2.0814, 0.7016, 2.0856, 1.7234, 0.4588, 0.3925, 0.5586, 1.1390,
         0.4954, 1.3265, 1.0008, 0.7692, 1.9938, 2.0773, 0.0851, 0.7397, 0.9467,
         1.3763, 0.8130, 1.6319, 0.6624, 0.9663, 0.9586, 0.9157, 0.4667, 0.9387,
         1.6578, 0.5540, 1.0880, 1.1384, 0.7308, 1.2564, 1.1406, 0.2934, 0.1940,
         0.2130, 0.5491, 0.4

In [14]:
import os.path as osp
import pickle

class CustomDataset(Dataset):

    def __init__(self, df, feature_dir):
        self.df = df
        self.feature_dir = feature_dir

    def __len__(self):
        return len(self.df)

    def aggregate_frame_features(self, frame_features: np.ndarray) -> np.ndarray:
        ans = np.mean(frame_features, axis = 0)
        if ans.ndim == 1:
            return ans
        else:
            return ans.squeeze()
        
    def load_features(self, feature_path):
        features = []
        with open(feature_path, 'rb') as f:
            while True:
                try:
                    _, frame_feature = pickle.load(f)
                    features.append(frame_feature)
                except EOFError:
                    break
        return features

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        vid = item['Id']
        label = item.get('Category', None)
        feature_path = osp.join(self.feature_dir, f'{vid}.pkl')
        frame_features = np.stack(self.load_features(feature_path))
        feature = self.aggregate_frame_features(frame_features)
        feature = torch.as_tensor(feature, dtype=torch.float)
        return feature, label

In [15]:
feat_dir = "/home/gyuseok/CMU/11775-hw2-handout/data/cnn3d"
val_dataset = CustomDataset(val_df, feat_dir)

In [16]:
train_dataset = CustomDataset(train_df, feat_dir)

In [17]:
train_dataset.__getitem__(0)[0].shape

torch.Size([512])

In [18]:
X2, X = 0,0
for idx in range(len(train_dataset)):
    target = train_dataset[idx][0]
    
    X2 += target**2
    X += target

    EX = X / len(train_dataset)
    VarX = X2/len(train_dataset) - EX**2


In [19]:
EX = X / len(train_dataset)

In [20]:
VarX = X2/len(train_dataset) - EX**2

VarX

tensor([0.2432, 0.1983, 0.3462, 0.3194, 0.3875, 0.3099, 0.4016, 0.1779, 0.4724,
        0.4910, 0.7908, 0.8887, 0.3600, 0.1911, 0.1705, 0.4684, 0.3028, 0.2925,
        0.3841, 0.2626, 0.3253, 0.2138, 0.3510, 0.1798, 0.3007, 0.3919, 0.3356,
        0.2557, 0.2599, 0.3318, 0.3711, 0.2393, 0.3530, 0.1969, 0.3113, 0.2960,
        0.3921, 0.6637, 0.5776, 0.1573, 0.3266, 0.5152, 0.5186, 0.4706, 0.3002,
        0.2957, 0.1957, 0.4191, 0.4254, 0.1917, 0.2182, 0.8444, 0.5059, 0.3345,
        0.2826, 0.1947, 0.2320, 0.7885, 0.2974, 0.4828, 0.2418, 0.4202, 0.3435,
        0.5882, 0.4034, 0.4268, 0.2594, 0.6309, 0.3150, 0.4568, 0.3780, 0.2856,
        0.2583, 0.6106, 0.3557, 0.4340, 0.7957, 0.4328, 0.1635, 0.3629, 0.3702,
        0.3376, 0.5342, 0.2725, 0.2277, 0.3452, 0.2742, 0.9035, 0.1957, 0.3610,
        0.3747, 0.3911, 0.3278, 0.2687, 0.2763, 0.5253, 0.3235, 0.2988, 0.4093,
        0.4619, 0.5414, 0.5521, 0.4002, 0.2382, 0.5199, 0.2572, 0.2356, 0.2041,
        0.4072, 0.5915, 0.4519, 0.5040, 

In [21]:
EX.shape

torch.Size([512])

In [22]:
(train_dataset[0][0] - EX) / VarX

tensor([ 1.6953e+00,  4.3419e-01,  1.2278e+00, -1.0744e+00,  1.4734e+00,
         1.0564e+00,  1.3286e+00,  5.9258e-01, -3.8540e-01, -5.7681e-01,
        -5.7945e-01,  7.0463e-01, -3.5866e-01, -2.1545e+00, -2.3029e+00,
        -1.3500e-01, -1.7866e+00,  2.3666e+00,  9.2221e-01, -1.3903e+00,
        -1.2324e+00, -6.0375e-01, -8.7069e-01, -1.6962e+00, -1.0063e+00,
         2.1785e+00,  4.9735e-01,  2.7075e+00,  5.4027e-01, -6.2745e-01,
         1.5071e+00,  1.9441e-01, -1.0122e+00,  1.2263e+00,  4.5862e-01,
        -6.3940e-01, -1.2133e+00,  5.0354e-01, -1.5248e+00,  5.5420e+00,
        -2.4111e-01, -5.8912e-02, -2.8916e-01, -1.3658e+00, -8.1303e-01,
        -3.0416e-01, -1.4508e+00, -1.3022e+00,  3.4832e-01,  1.3436e+00,
        -1.3038e+00, -4.5703e-01,  8.0051e-01, -1.0651e+00,  2.3194e+00,
        -2.0459e-01, -7.1495e-01, -6.5683e-01, -1.9610e+00,  2.4918e+00,
        -1.5811e+00,  1.1189e-01, -6.8278e-01, -5.5700e-01, -1.3021e+00,
         6.1971e-01,  2.5323e+00,  1.2120e+00, -8.1

In [23]:
a = None
b = None

In [24]:
if not (a or b):
    print(1)

1


In [25]:
dataset1 = CustomTrain(np.array(feat_list), np.array(label_list))

In [26]:
train_dataset

In [27]:
train_dataset.__getitem__(0)[1]
#dataset1.__getitem__(0)[1]

8

In [28]:
from torch.utils.data import ConcatDataset


c_dataset = ConcatDataset([train_dataset, dataset1])

In [29]:
a = torch.randn(512)
b = torch.randn(1024)
torch.concat([a,b]).shape

torch.Size([1536])

In [30]:
from torch.utils.data import Dataset

class ConcatData(Dataset):
    def __init__(self, datasets):
        self.datasets = datasets
        
    def __len__(self):
        return len(self.datasets[0])

    def __getitem__(self,idx):
        x_data = [dataset[idx][0] for dataset in self.datasets]
        x_data = torch.concat(x_data)
        x_data = torch.FloatTensor(x_data)
        
        
        y_data = self.datasets[0][idx][1]
        y_data = torch.tensor(y_data)

        return x_data, y_data

In [31]:
c_data = ConcatData([train_dataset, dataset1])

In [32]:
c_data.__getitem__(3)[0].shape

torch.Size([1536])

In [35]:
a = (5,2)
type(a) == tuple

True

In [41]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size = 750)

In [47]:
t = iter(train_loader)

In [57]:
next(t)

StopIteration: 

In [61]:
x_data = [torch.randn(750,32),torch.randn(750,32) ]
torch.concat(x_data, dim = 1).shape

torch.Size([750, 64])